In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop
import os
from datetime import date

## API data response

<pre>
dict( 
    data : list(one element is one earning call metada)
    included : "don't care for now"
    meta : "don't care for now"
        )
</pre>
One earning call metadata is show below 

In [2]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 400
date_of_request = date.today().strftime("%Y-%m-%d") #e.g. "2022-04-28"
max_return_per_request = 50
api_data_responses = []
for page_number in range(1,nbr_earning_call_metada//max_return_per_request + 1):
    url = "https://seekingalpha.com/api/v3/articles"+"?cacheBuster="\
          +str(date_of_request)\
          +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"\
          +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="\
          +str(max_return_per_request)\
          +"&page[number]="+str(page_number)
    # the chrome driver depend on which version of chrome you are running specify the good one
    executable_path = "./chromedriver_linux"
    service = Service(executable_path)
    option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service,options=option)
    driver.get(url)
    api_json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
    api_data_responses.extend(json.loads(api_json_response)["data"])
print(url)
#a quick loock of the result to check if we get results are get captcha/bot error 
print(api_json_response[0:100])

https://seekingalpha.com/api/v3/articles?cacheBuster=2022-05-09&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]=50&page[number]=8
{"data":[{"id":"4508470","type":"transcript","attributes":{"publishOn":"2022-05-07T13:14:02-04:00","


In [3]:
api_data_responses[0]

{'id': '4508949',
 'type': 'transcript',
 'attributes': {'publishOn': '2022-05-09T06:41:05-04:00',
  'isLockedPro': False,
  'commentCount': 0,
  'gettyImageUrl': None,
  'themes': {},
  'title': 'Sprague Resources LP (SRLP) CEO David Glendon on Q1 2022 Results - Earnings Call Transcript'},
 'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}},
  'sentiments': {'data': []},
  'primaryTickers': {'data': [{'id': '113512', 'type': 'tag'}]},
  'secondaryTickers': {'data': []},
  'otherTags': {'data': [{'id': '96991', 'type': 'tag'},
    {'id': '17810', 'type': 'tag'},
    {'id': '49', 'type': 'tag'},
    {'id': '586376', 'type': 'tag'},
    {'id': '326', 'type': 'tag'},
    {'id': '17823', 'type': 'tag'}]}},
 'links': {'self': '/article/4508949-sprague-resources-lp-srlp-ceo-david-glendon-on-q1-2022-results-earnings-call-transcript'}}

In [4]:

id_and_path_to_earning_calls = [ (earning_call_metada["id"] ,earning_call_metada["links"]["self"]) 
                     for earning_call_metada in api_data_responses]
# ""check url
id_and_path_to_earning_calls[0:2]

[('4508949',
  '/article/4508949-sprague-resources-lp-srlp-ceo-david-glendon-on-q1-2022-results-earnings-call-transcript'),
 ('4508948',
  '/article/4508948-american-financial-group-inc-afg-q1-2022-results-earnings-call-transcript')]

## download all not yet download html from url

In [8]:
domain_name = 'https://seekingalpha.com'
data_path = "data/article"
need_to_download = []
if not os.path.exists(data_path):
    os.makedirs(data_path)
    print("The new directory is created!")
for id_earning_call, path_to_earning_call in tqdm(id_and_path_to_earning_calls):
    if not os.path.isfile(data_path+"/"+ id_earning_call +".html"):
        # not in local directory
        need_to_download.append((id_earning_call, path_to_earning_call))
print(f"Need to download {len(need_to_download)} html file(s)")
for id_earning_call, path_to_earning_call in tqdm(need_to_download):
    with open(data_path+"/"+ id_earning_call+".html", "x") as file:
            url = domain_name + path_to_earning_call
            html_transcript = requests.get(url).text
            file.write(html_transcript)
    

The new directory is created!


100%|██████████████████████████████████████| 400/400 [00:00<00:00, 31335.85it/s]


Need to download 400 html file(s)


100%|█████████████████████████████████████████| 400/400 [04:31<00:00,  1.48it/s]


## Check last file got downloaded, i.e. captcha/bot error 

In [9]:
BeautifulSoup(html_transcript).get_text()[:2000]

"Backblaze Inc. (BLZE) CEO Gleb Budman on Q1 2022 Results - Earnings Call Transcript | Seeking Alpha                    Seeking Alpha - Go to HomepageTrendingMy PortfolioMy AuthorsTop StocksLatest NewsMarketsStock IdeasDividendsETFsEducationWatch & ListenTranscriptsTechnologyBackblaze Inc. (BLZE) CEO Gleb Budman on Q1 2022 Results - Earnings Call TranscriptMay 07, 2022 8:46 AM ETBackblaze, Inc. (BLZE)SA Transcripts126.58K FollowersFollowBackblaze, Inc. (NASDAQ:BLZE) Q1 2022 Earnings Conference Call May 5, 2022 4:30 PM ET Company Participants James Kisner – Vice President of Investor Relations Gleb Budman – Co-Founder, Chief Executive Officer & Chairperson Frank Patchel – Chief Financial Officer Conference Call Participants Ittai Kidron – Oppenheimer Erik Suppiger – JMP Securities Eric Martinuzzi – Lake Street Capital Billy Fitzsimmons – William Blair Zach Cummins – B. Riley Operator Good afternoon, ladies and gentlemen, thank you for standing by. And welcome to Backblaze's First Quarte